# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Link to dataset from kaggle: https://www.kaggle.com/jessemostipak/hotel-booking-demand
hotel_df = pd.read_csv('C:/Users/keger/Documents/Thinkful_dataScience/supervisedLearning_knn/knn_challenge/hotel_bookings.csv')

# Data Cleaning

In [2]:
hotel_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
hotel                             119390 non-null object
is_canceled                       119390 non-null int64
lead_time                         119390 non-null int64
arrival_date_year                 119390 non-null int64
arrival_date_month                119390 non-null object
arrival_date_week_number          119390 non-null int64
arrival_date_day_of_month         119390 non-null int64
stays_in_weekend_nights           119390 non-null int64
stays_in_week_nights              119390 non-null int64
adults                            119390 non-null int64
children                          119386 non-null float64
babies                            119390 non-null int64
meal                              119390 non-null object
country                           118902 non-null object
market_segment                    119390 non-null object
distribution_channel              119390 n

So, we can see that there are some columns with a significant number of null values, specififcally the company and the agent column, so we can drop those columns. The country and children columns have less nulls, so we will just remove the rows where those are null, there is enough data otherwise.

In [4]:
# drop company and agent columns 
hotel_df.drop(['company', 'agent'], axis=1, inplace=True)

# drop rows with null values in the country and children columns
hotel_df.dropna(axis=0, subset=['country', 'children'], inplace=True)

# drop reservation date column, we will generalize over the month
hotel_df.drop(['reservation_status_date'], axis=1, inplace=True)

# drop country column
hotel_df.drop(['country'], axis=1, inplace=True)

hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118898 entries, 0 to 119389
Data columns (total 28 columns):
hotel                             118898 non-null object
is_canceled                       118898 non-null int64
lead_time                         118898 non-null int64
arrival_date_year                 118898 non-null int64
arrival_date_month                118898 non-null object
arrival_date_week_number          118898 non-null int64
arrival_date_day_of_month         118898 non-null int64
stays_in_weekend_nights           118898 non-null int64
stays_in_week_nights              118898 non-null int64
adults                            118898 non-null int64
children                          118898 non-null float64
babies                            118898 non-null int64
meal                              118898 non-null object
market_segment                    118898 non-null object
distribution_channel              118898 non-null object
is_repeated_guest                 118898 n

Now all the columns have the same amount of data in them, so we can proceed with analysis. For the purpose of this model, the target variable will be whether the hotel booking is cancelled. 

#  Exploratory Data Analysis and Feature Engineering

In [5]:
non_numeric_columns = hotel_df.select_dtypes(['object']).columns
print(non_numeric_columns)

for col in non_numeric_columns :
    print(col, hotel_df[col].unique())

Index(['hotel', 'arrival_date_month', 'meal', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'customer_type', 'reservation_status'],
      dtype='object')
hotel ['Resort Hotel' 'City Hotel']
arrival_date_month ['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June']
meal ['BB' 'FB' 'HB' 'SC' 'Undefined']
market_segment ['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary' 'Groups'
 'Aviation']
distribution_channel ['Direct' 'Corporate' 'TA/TO' 'Undefined' 'GDS']
reserved_room_type ['C' 'A' 'D' 'E' 'G' 'F' 'H' 'L' 'B' 'P']
assigned_room_type ['C' 'A' 'D' 'E' 'G' 'F' 'I' 'B' 'H' 'L' 'K' 'P']
deposit_type ['No Deposit' 'Refundable' 'Non Refund']
customer_type ['Transient' 'Contract' 'Transient-Party' 'Group']
reservation_status ['Check-Out' 'Canceled' 'No-Show']


In [6]:
numeric_columns = hotel_df.select_dtypes(['int64', 'float64']).columns
print(numeric_columns)

Index(['is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests'],
      dtype='object')


In [7]:
hotel_dummies = pd.get_dummies(hotel_df, drop_first=True)
hotel_dummies.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Check-Out,reservation_status_No-Show
0,0,342,2015,27,1,0,0,2,0.0,0,...,0,0,0,0,0,0,1,0,1,0
1,0,737,2015,27,1,0,0,2,0.0,0,...,0,0,0,0,0,0,1,0,1,0
2,0,7,2015,27,1,0,1,1,0.0,0,...,0,0,0,0,0,0,1,0,1,0
3,0,13,2015,27,1,0,1,1,0.0,0,...,0,0,0,0,0,0,1,0,1,0
4,0,14,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,1,0,1,0


In [8]:
hotel_dummies.corrwith(hotel_dummies['is_canceled']).sort_values(ascending=False).head(20)

is_canceled                     1.000000
deposit_type_Non Refund         0.481349
lead_time                       0.291994
market_segment_Groups           0.221538
distribution_channel_TA/TO      0.174134
customer_type_Transient         0.134473
reservation_status_No-Show      0.131542
previous_cancellations          0.109922
adults                          0.058381
days_in_waiting_list            0.054016
adr                             0.046199
meal_FB                         0.038729
arrival_date_month_June         0.028696
stays_in_week_nights            0.024110
arrival_date_month_May          0.017646
arrival_date_year               0.016412
arrival_date_month_September    0.013089
arrival_date_week_number        0.007465
arrival_date_month_October      0.006454
reserved_room_type_H            0.005355
dtype: float64

In [9]:
keep_list = ['deposit_type_Non Refund', 'lead_time', 'market_segment_Groups', 'distribution_channel_TA/TO',
            'customer_type_Transient', 'reservation_status_No-Show', 'previous_cancellations', 'adults', 
            'days_in_waiting_list', 'adr']

# KNN Model

In [11]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=20)
X = hotel_dummies[keep_list]
Y = hotel_dummies['is_canceled']
knn.fit(X, Y)

from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Unweighted Accuracy: -66.63 (+/- 251.70)


# OLS Model

In [12]:
import statsmodels.api as sm

X = hotel_dummies[keep_list]
Y = hotel_dummies['is_canceled']

X = sm.add_constant(X)

C:\Users\keger\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2)

results = sm.OLS(Y_train, X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            is_canceled   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     4065.
Date:                Fri, 27 Mar 2020   Prob (F-statistic):               0.00
Time:                        12:25:55   Log-Likelihood:                -48814.
No. Observations:               95118   AIC:                         9.765e+04
Df Residuals:                   95107   BIC:                         9.775e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -0.0808      0.006    -13.784      0.000      -0.092      -0.069
deposit_type_Non Refund        0.5795      0.005    114.460      0.000       0.570       0.589
lead_time                      0.0006   1.42e-05     41.788      0.000       0.001       0.001
market_segment_Groups          0.0546      0.004     12.196      0.000       0.046       0.063
distribution_channel_TA/TO     0.1251      0.004     35.274      0.000       0.118       0.132
customer_type_Transient        0.1248      0.003     35.695      0.000       0.118       0.132
reservation_status_No-Show     0.7414      0.013     56.168      0.000       0.716       0.767
previous_cancellations         0.0298      0.002     18.302      0.000       0.027       0.033
adults                         0.0184      0.002      7.537      0.000       0.014       0.023
days_in_waiting_list          -0.0003   7.63e-05     -3.369      0.001      -0.000      -0.000
adr                            0.0007   2.71e-05     25.167      0.000       0.001       0.001
==============================================================================
Omnibus:                    13156.553   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15769.499
Skew:                           0.956   Prob(JB):                         0.00
Kurtosis:                       2.430   Cond. No.                     1.71e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Based on the two regressions, it seems that the OLS model would perform better than KNN model at predicting whether a booking is likely to be canceled. 